In [30]:
import numpy as np
from scipy.constants import c
from numba import njit
from senior_sem import generate_satellite,generate_transmission

In [31]:
import numpy as np
from datetime import datetime
from scipy.constants import c
a = 6_378_137.00
e_sq =  6.69437999014 * 10**-3
def v(phi):
    """Make sure to convert to degrees first"""
    return a/np.sqrt(1-e_sq*np.sin(phi)**2)
def generate_transmission(lat,lon):
    time_of_transmission=np.datetime64(datetime.now())+np.timedelta64(np.random.randint(10**9),"ns")
    randlat= lat+np.random.randint(5)+np.random.rand()
    randlon=lon+np.random.randint(5)+np.random.rand()
    return randlat,randlon,time_of_transmission
def wgs_84_geo_to_cartes(lat,lon,alt):
    lat,lon=np.deg2rad([lat,lon])
    x=(v(lat)+alt)*np.cos(lat)*np.cos(lon)
    y=(v(lat)+alt)*np.cos(lat)*np.sin(lon)
    z=(v(lat)*(1-e_sq)+alt)*np.sin(lat)
    return x,y,z
def cartes_tuple_distance(tuple1,tuple2):
    x1,y1,z1=tuple1
    x2,y2,z2=tuple2
    return np.sqrt((x1-x2)**2+(y1-y2)**2+(z1-z2)**2)
def generate_satellite(trans_tuple):
    lat,lon,time =trans_tuple
    randlat = lat+np.random.randint(40)+np.random.rand()
    randlon = lon+np.random.randint(40)+np.random.rand()
    alt = 50*10**6 + np.random.randint(10**6)
    dist = cartes_tuple_distance(wgs_84_geo_to_cartes(lat,lon,0),
                                 wgs_84_geo_to_cartes(randlat,randlon, alt))
    # return dist
    time_of_travel_rounded=np.timedelta64(int(np.round(dist/c,9)*10**9),"ns")
    time_of_arrival = time + time_of_travel_rounded
    return {"coords":wgs_84_geo_to_cartes(randlat,randlon, alt),"TOA":time_of_arrival}#,randlat,randlon

In [32]:
a = 6_378_137.00
f = 1/298.257223563
e_sq =  2*f-f**2
trans1 = generate_transmission(4,5)
sat1 = generate_satellite(trans_tuple=trans1)
sat2 = generate_satellite(trans_tuple=trans1)

In [33]:
def v(phi):
    return a/np.sqrt(1-e_sq*np.sin(phi)**2)

In [34]:
def ds(phi,gamma,x,y,z):
    return np.sqrt((v(phi)*np.cos(phi)*np.cos(gamma)-x)**2 
                   + (v(phi)*np.cos(phi)*np.sin(gamma)-y)**2 
                   + ((1-e_sq)*v(phi)*np.sin(phi)-z)**2)

In [35]:
def TDOA_geo(gamma,phi,s0,s1):
    ta0=s0["TOA"]
    ta1=s1["TOA"]
    x0,y0,z0=s0["coords"]
    x1,y1,z1=s1["coords"]

    return (ta0-ta1)/np.timedelta64(1,"s")+(ds(phi,gamma,x1,y1,z1)-ds(phi,gamma,x0,y0,z0))/c

In [36]:
TDOA_geo(6,6,sat1,sat2)

-0.0014409090588405598

In [37]:
from scipy.optimize import newton

In [39]:
trans1

(8.70886338901948,
 5.028510045090148,
 numpy.datetime64('2024-01-04T11:41:44.285182845'))

In [40]:
phi = np.arange(-89,90,.1)
phi.shape
theta = np.arange(-89,90,.1)
theta.shape
rp,rt=np.meshgrid(phi, theta)

In [41]:
TDOA_geo(phi,theta,sat1,sat2)

array([-0.00434361, -0.00372021, -0.00317259, ...,  0.01275249,
        0.01317048,  0.0133872 ])

TODO make a sympy version of TDOA so that it can be differentiated with respect to latitude

In [42]:
def diff_tdoa(gamma,phi,s0,s1):
    x_0,y_0,z_0=s0["coords"]
    x_1,y_1,z_1=s1["coords"]
    return (-(a*(a*np.sin(gamma)*np.cos(phi)/np.sqrt(-e_sq*np.sin(phi)**2 + 1) - y_0)*np.cos(gamma)*np.cos(phi)/np.sqrt(-e_sq*np.sin(phi)**2 + 1) - a*(a*np.cos(gamma)*np.cos(phi)/np.sqrt(-e_sq*np.sin(phi)**2 + 1) - x_0)*np.sin(gamma)*np.cos(phi)/np.sqrt(-e_sq*np.sin(phi)**2 + 1))/np.sqrt((a*(1 - e_sq)*np.sin(phi)/np.sqrt(-e_sq*np.sin(phi)**2 + 1) - z_0)**2 + (a*np.sin(gamma)*np.cos(phi)/np.sqrt(-e_sq*np.sin(phi)**2 + 1) - y_0)**2 + (a*np.cos(gamma)*np.cos(phi)/np.sqrt(-e_sq*np.sin(phi)**2 + 1) - x_0)**2) + (a*(a*np.sin(gamma)*np.cos(phi)/np.sqrt(-e_sq*np.sin(phi)**2 + 1) - y_1)*np.cos(gamma)*np.cos(phi)/np.sqrt(-e_sq*np.sin(phi)**2 + 1) - a*(a*np.cos(gamma)*np.cos(phi)/np.sqrt(-e_sq*np.sin(phi)**2 + 1) - x_1)*np.sin(gamma)*np.cos(phi)/np.sqrt(-e_sq*np.sin(phi)**2 + 1))/np.sqrt((a*(1 - e_sq)*np.sin(phi)/np.sqrt(-e_sq*np.sin(phi)**2 + 1) - z_1)**2 + (a*np.sin(gamma)*np.cos(phi)/np.sqrt(-e_sq*np.sin(phi)**2 + 1) - y_1)**2 + (a*np.cos(gamma)*np.cos(phi)/np.sqrt(-e_sq*np.sin(phi)**2 + 1) - x_1)**2))/c

In [44]:
newton(TDOA_geo,0,args=(5.02851,sat1,sat2),fprime=diff_tdoa)

RuntimeError: Failed to converge after 1000 iterations, value is -0.24988519973253087.